In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re 

In [2]:
data = pd.read_excel('ISO.xlsx', header = None, names = ['standarts', 'title'])
data['year'] = np.zeros(len(data))
data

,standarts,title,year
0,ISO 15926-3,NaN,0.0
1,ISO 20816-1,NaN,0.0
2,ISO 5807,NaN,0.0
3,ISO 6385,NaN,0.0
4,ISO 7250-1,NaN,0.0
5,ISO/IEC 29110-5-1-2,NaN,0.0
6,ISO/IEC 27001,NaN,0.0
7,ISO/IEC/IEEE 26512,NaN,0.0


In [3]:
n  = len(data)
for i in range(n):
    name_number = data['standarts'][i]
    url = 'https://www.beuth.de/de/erweiterte-suche/272754!search?alx.searchType=complex&searchAreaId=1&query='+name_number+'&facets%5B276612%5D=&hitsPerPage=50'
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    
    start = str(soup.find('a', class_ = 'bwr-link bwr-card__title-link')).find('/')
    end = str(soup.find('a', class_ = 'bwr-link bwr-card__title-link')).find('">')
    link = str(soup.find('a', class_ = 'bwr-link bwr-card__title-link'))[start:end]
    
    url = 'https://www.beuth.de'+link
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    
    s = str(soup.find('title')).replace("<title>", "").replace(' - Beuth.de</title>', '')
    data.loc[i,'title'] = s

    match_20 = re.search("20\d\d", s)
    year_20 = (int(match_20[0]) if match_20 else 0)
    
    match_19 = re.search("19\d\d", s)
    year_19 = (int(match_19[0]) if match_19 else 0)
        

    data.loc[i,'year'] = max(year_20, year_19)
            
            

In [4]:
data

,standarts,title,year
0,ISO 15926-3,ISO/TS 15926-3 - 2009-05,2009.0
1,ISO 20816-1,ISO 20816-1 - 2016-11,2081.0
2,ISO 5807,ISO 5807 - 1985-02,1985.0
3,ISO 6385,ISO 6385 - 2016-09,2016.0
4,ISO 7250-1,ISO 7250-1 - 2017-08,2017.0
5,ISO/IEC 29110-5-1-2,CAN/CSA-ISO/IEC TR 29110-5-1-2 - 2018-01-01,2018.0
6,ISO/IEC 27001,ISO/IEC 27001 - 2013-10,2013.0
7,ISO/IEC/IEEE 26512,ISO/IEC/IEEE 26512 - 2018-06,2018.0


In [5]:
data.to_excel('ISO_out.xlsx')